### Линейная регрессия

Вы ищете работу. На сайте hh.ru представлено описание вакансии, но не указан размер оплаты.

Цель: построить модель, которая по описанию вакансии предсказывает заработную плату,
которую предполагает работодатель.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.inspection import permutation_importance
import scipy.stats as stats

In [3]:
def deprecate_of(title):
    if ' of ' in title:
        parts = title.split(' of ')
        return f"{parts[1].strip()} {parts[0].strip()}"
    return title

def preprocess_strings(data):
    df = data.copy(deep=True)
    df['Job Title'] = df['Job Title'].str.lower()
    df['Job Title'].replace("human resources", "hr", inplace=True)
    df['Job Title'] = df['Job Title'].apply(deprecate_of)
    df['Job Title'].replace("representative", "rep", inplace=True)
    df['Job Title'].replace(["senior", "junior"], "", inplace=True)
    df['Gender'] = df['Gender'].str.lower()
    df['Education Level'] = df['Education Level'].str.lower() 
    return df

def preprocess(data):
    df = data.copy(deep=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df = preprocess_strings(df)
    return df

def get_features_target(data):
    df = data.copy(deep=True)
    target_col = df.pop("Salary")
    df["Salary"] = target_col
    features = df.iloc[:, :-1]
    target = df.iloc[:, -1]
    return features, target

def encode_strings(data, encoders = None):
        features = ['Gender', 'Education Level', 'Job Title']
        df = data.copy(deep=True)
        encs = encoders
        if encoders is None:
            encs = []
            for feature in features:
                enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
                enc.fit(df[[feature]])
                encs.append(enc)
        for i, feature in enumerate(features):
            enc = encs[i]
            df[feature] = enc.transform(df[[feature]])
        return df, encs

In [4]:
path = "D:/python/mathModelling/hw4/Salary Data.csv"

data = pd.read_csv('D:/python/mathModelling/hw4/Salary Data.csv')

data = preprocess(data)

data, encoders = encode_strings(data)

features, target = get_features_target(data)

Обучим и протестируем модель

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [7]:
r_squared = model.score(X_test, y_test)
print(f'Коэффициент детерминации R^2: {r_squared:.3f}')
y_pred = model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print('Корень среднеквадратичной ошибки: ', np.round(rmse))

Коэффициент детерминации R^2: 0.881
Корень среднеквадратичной ошибки:  15940.0


Посмотрим, какие параметры вносят больший вклад предсказание при помощи случайного леса


In [8]:
result = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=0)

importance_df = pd.DataFrame({'Feature': X_test.columns, 'Importance': result.importances_mean})
print(importance_df)

               Feature  Importance
0                  Age    0.339674
1               Gender    0.010088
2      Education Level    0.088905
3            Job Title    0.000604
4  Years of Experience    0.353721


Больший вклад вносят возраст и стаж. В принципе эти две величины являются коррелирующими величинами, у молодых людей чисто физически не может быть большой стаж. Также стоит отметить, что в идеале нужно лучшим образом преобразовать должности. Например, можно разделить должность на сферу деятельности (продажи, разработка и т.д.) и выполняемую роль (менеджер, HR и др.). Но такая обработка кажется затруднительной с той позиции, что необходимо вручную просмотреть каждую позицию, чтобы выделить это в сферу или роль. Также с некоторыми позициями возникают сложности: например, в строке 50 есть кандидат "38,Male,PhD,Senior Scientist,11,120000". Допустим, роль Scientist, но сфера также Science? Всё же такая позиция является некорректной в плане полноты данных.

Оценим доверительный интервал нашей модели с уровнем значимости $\alpha = 5%$

Стандартная ошибка линейной модели выражается формулой

$$
    SE = \sqrt{\dfrac{\sum\left( y_i - \hat{y}_i \right)^2}{n-k-1}}
$$

В случае линейной модели $k=1$

Размер тестовой выборки $n = 65$

Доверительный интервал:

$$
    \hat{y} \pm t_{0.975,63} \cdot SE
$$

In [9]:
k = 1
n = len(X_test)
alpha = 0.05
res = np.sqrt(np.sum((y_test - y_pred)**2) / n - k - 1)

t_crit = stats.t.ppf(1 - alpha / 2, n - k - 1)



Теперь применим нашу модель к предложенным данным

In [10]:
candidate = {'Age' : [40], 'Gender' : "male", 'Education Level' : "Master's", "Job Title" : "HR Project Manager", "Years of Experience" : 10}

df = pd.DataFrame(candidate)

df = preprocess(df)

df, _ = encode_strings(df, encoders)

prediction = np.round(model.predict(df)[0])
salary_low = np.round(prediction - t_crit * res)
salary_high = np.round(prediction + t_crit * res)

print(f"Пределы зарплаты: от {salary_low} до {salary_high}")

Пределы зарплаты: от 86354.0 до 150062.0


И для второго примера:

In [11]:
candidate = {'Age' : [29], 'Gender' : "female", 'Education Level' : "Master's", "Job Title" : "Web Developer", "Years of Experience" : 3}

df = pd.DataFrame(candidate)

df = preprocess(df)

df, _ = encode_strings(df, encoders)

prediction = np.round(model.predict(df)[0])
salary_low = np.round(prediction - t_crit * res)
salary_high = np.round(prediction + t_crit * res)

print(f"Пределы зарплаты: от {salary_low} до {salary_high}")

Пределы зарплаты: от 26511.0 до 90219.0


Заметим, что разброс получается довольно большим (относительно), так как интервал получился фиксированного размера и зависит только от тренировочных данных. Относительный разброс получился небольшим по той причине, что я указал минимальный требуемый стаж и +/- реалистичный возраст кандидата для такого стажа

### Замечание

При извлечении данных из тестовой выборки приходится проявлять фантазию по той причине, что только небольшой набор фактической информации можно извлечь: должность и опыт работы (если указан). Остальные параметры как пол, возраст, образование задаются произвольным. Но важно отметить, что возраст коррелирует со стажем, поэтому произвольно указанный возраст будет в значительной мере влиять на данные.

Также, кажется, что формат выдаваемого тестового датасета в некой мере некорректен в том смысле, что будто бы нет смысла в csv формате указывать данные по типу местоположения работы и срок контракта, ведь эта информация нигде не фигурировала в тренировочных данных. Тем самым изначально можно уменьшить "размер" тестовой выборки, если сразу убрать излишнюю информацию, которая заведомо не будет использоваться в регрессии